Define some variables and input files

In [ ]:
import importlib.util
import os

from svgpathtools import Document, Path, Line, QuadraticBezier, CubicBezier, Arc, is_bezier_path, is_bezier_segment, is_path_segment, svg2paths, bpoints2bezier

import numpy as np
import svg2contours as svg2contours

import importlib
importlib.reload(svg2contours)


data_path = "../../../data/contours/svg"
svg_file = "shapes.svg"
# svg_file = "transform.svg"

# data_path = "."
# svg_file = "ellipse.svg"
# svg_file = "mdn_arc_example.svg"
# svg_file = "four_arcs.svg"


input_file = os.path.join(data_path, svg_file)
# print(os.path.abspath(input_file))
# print(os.listdir(data_path))

output_file = "drawing.mesh"

doc = Document(input_file)
paths = doc.paths()

for k,v in doc.tree.getroot().attrib.items():
    print(k,v)

coordinate_transform = svg2contours.get_root_transform(doc)


Convert paths to mfem mesh of cubic Bezier segments using ASCII output

In [ ]:
print(paths)

import importlib
importlib.reload(svg2contours)

mfem_data = svg2contours.MFEMData()
print(f"""Creating mfem_date -- {mfem_data.elem_cnt=}""")


for p_idx, p in enumerate(paths):
    print(f"""reading {p_idx=} {p=} \n w/ {p.d()=}""")

    is_d_path = 'd' in p.element.keys()
    attrib = p_idx + 1

    reverse_paths = True if np.linalg.det(coordinate_transform) < 0 else False

    if not all(map(is_path_segment, p)):
        continue

    for seg_idx, seg in enumerate(p):

        if isinstance(seg, Arc) and seg.large_arc and is_d_path:
            # split large elliptical arcs for easier processing
            # this simplifies the derivation of the internal control points
            # in `arc_to_cubic` algorithm
            arc1,arc2 = seg.split(.5)
            
            cubic,weights = svg2contours.segment_as_cubic(arc1, reverse_paths)
            xformed_cubic = svg2contours.transform_cubic(cubic, coordinate_transform)
            mfem_data.add_cubic_bezier(xformed_cubic, weights, attrib)
            
            cubic,weights = svg2contours.segment_as_cubic(arc2, reverse_paths)
            xformed_cubic = svg2contours.transform_cubic(cubic, coordinate_transform)
            mfem_data.add_cubic_bezier(xformed_cubic, weights, attrib)
        else:
            cubic,weights = svg2contours.segment_as_cubic(seg, reverse_paths)
            xformed_cubic = svg2contours.transform_cubic(cubic, coordinate_transform)
            mfem_data.add_cubic_bezier(xformed_cubic, weights, attrib)

mfem_data.write_file(output_file)

